# Statistical Tests (VIF and P-value) for Feature Sets

### VIF

Variance Inflation Factor (VIF) is a measure used to assess multicollinearity in regression analysis. It quantifies how much the variance of the estimated regression coefficient is increased due to the correlation between predictor variables. A high VIF value indicates a strong correlation between predictor variables, which can lead to unreliable and unstable regression coefficient estimates. In other words, VIF helps identify predictors that are highly correlated with each other, which can cause issues in the interpretation and reliability of the regression model.

- 1 = not correlated.
- Between 1 and 5 = moderately correlated.
- Greater than 5 = highly correlated.

### P-value

P-value is used to determine the statistical significance of each predictor (or independent variable) in the model. The p-value is a measure of the probability that an observed difference could have occurred just by random chance. If the p-value is less than a certain significance level (commonly 0.05), then we conclude that the predictor is statistically significant.

- Small is better as it indicates that a result did not take place by chance
- Smaller means the null hypothesis (that there is no relationship exists between two variables)
- smaller than 0.05 indicates significance

In [129]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sys
from datasets import load_from_disk

sys.path.append("../")
import utils.features

In [130]:
PROJECT_DIR = os.path.dirname(os.getcwd())
STATS_DIR = os.path.join(PROJECT_DIR, 'classification/stats')
FEATURES_DIR = os.path.join(PROJECT_DIR, 'feature_extraction/features/')
RESPONSES_DIR = os.path.join(PROJECT_DIR, 'responses/')
SPLITS_DIR = os.path.join(PROJECT_DIR, "classification/split_datasets/coqa")
PROJECT_DIR

'/mount/studenten-temp1/users/dpgo/xai-thesis'

In [131]:
existing_splits = True

def load_x_y(features_filename, responses_filename):
    
    # load features (x)
    features_df = utils.features.load_features(features_filename)

    # load labels (y)
    labels_df = utils.features.load_labels(responses_filename)

    # merge
    data_df = utils.features.merge_and_filter(features_df, labels_df)

    # for testing: 15 rows, 15 columns
    #data_df = data_df.iloc[-15:, -15:] # tmp

    # normalize + select features
    data_df = manipulate_features(data_df)

    print(f"Instances: {len(data_df)}\nFeatures: {len(data_df.columns)-1}")

    return data_df


def manipulate_features(data_df):
    # normalize features
    data_df = utils.features.scale_features(
        data_df, 
        scaler=MinMaxScaler(
            feature_range=(0, 1)
        )
    )
    
    return data_df


def prepare_splits(data_df, test_size=0.2, random_state=1):

    if existing_splits: 
        print("\nUsing existing splits")
        # use exact same split as in the SPLITS_DIR
        raw_dataset = load_from_disk(SPLITS_DIR)
        # eg. train_df contains instances in raw_dataset['train']
        # based on the pandas_idx in raw_dataset['train']['pandas_idx']
        train_df = data_df[data_df.index.isin(raw_dataset['train']['pandas_idx'])]
        test_df = data_df[data_df.index.isin(raw_dataset['test']['pandas_idx'])]
    else:
        print("\nCreating new splits")
        train_df, test_df = train_test_split(
            data_df, 
            test_size=test_size, 
            random_state=random_state
        )

    X_train = train_df.drop(columns=['outcome'])
    y_train = train_df['outcome']

    X_test = test_df.drop(columns=['outcome'])
    y_test = test_df['outcome']

    print('X_train shape:', X_train.shape)
    print('y_train shape:', y_train.shape)
    print('X_test shape:', X_test.shape)
    print('y_test shape:', y_test.shape)

    return X_train, y_train, X_test, y_test

## col-RFE feature set

- 300 features

1. remove collinear features
2. Recursive Feature Elimination

In [132]:
feature_set_source = os.path.join(STATS_DIR, "12091031_col-rfe_all_10000.json")
with open(feature_set_source) as f:
    stats = json.load(f)
    feature_set = [f for f in stats['coefficients'].keys()]

In [133]:
data_df = load_x_y(
    features_filename=os.path.join(FEATURES_DIR, '12091031_all_features.csv.gz'),
    responses_filename=os.path.join(RESPONSES_DIR, '12091031_parsed_turbo_10000_eval.jsonl')
)
X_train, y_train, X_test, y_test = prepare_splits(data_df, test_size=0.2, random_state=1)

Instances: 10000
Features: 2371

Using existing splits
X_train shape: (8000, 2371)
y_train shape: (8000,)
X_test shape: (1000, 2371)
y_test shape: (1000,)


In [134]:
# Only use features in feature set
X_train = X_train[feature_set]
X_test = X_test[feature_set]

print('X_train shape:', X_train.shape, 'X_test shape:', X_test.shape)

X_train shape: (8000, 300) X_test shape: (1000, 300)


In [135]:
constant_features = X_train.columns[X_train.nunique() == 1] # constant features 

X_train = X_train.drop(columns=constant_features)
X_test = X_test.drop(columns=constant_features)

X_train.shape, X_test.shape

((8000, 299), (1000, 299))

In [136]:
# Check for multicollinearity - there should be none (already removed highly correlated features)
corr_matrix = X_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]
to_drop

[]

In [137]:
coefficients = stats['coefficients']

threshold = 0.1
large_coefficients = {k: v for k, v in coefficients.items() if abs(v) > threshold}

In [138]:
len(coefficients), len(large_coefficients)

(300, 297)

In [139]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# calculate VIF and p-value

# a feature is robust if it has a low p-value and low VIF (and a large coefficient)
robust_features = {} # individual features and interactions

for feature, coef in large_coefficients.items():

    X = sm.add_constant(X_train[[feature]]) # stats models allows us to get the p-values
    X.reset_index(drop=True, inplace=True) # avoid index mismatch error
    y = y_train.reset_index(drop=True) # avoid index mismatch error
    
    sm_model = sm.Logit(y, X)
    result = sm_model.fit(method='lbfgs', maxiter=1000, disp=False)

    p_value = result.pvalues[1]
    
    vif = variance_inflation_factor(X.values, 1) # idx of col to check (0: constant, 1: feature since we add them one by one)

    if p_value < 0.05 and vif < 5: # robust and significant
        robust_features[feature] = {
            'coefficient': round(coef, 5), 
            'p_value': round(p_value, 5), 
            'vif': round(vif, 5)
        }

In [140]:
len(robust_features.keys())

108

In [141]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    penalty='l2', 
    C=1.0, 
    solver='lbfgs', 
    max_iter=1000, 
    random_state=1
)

model.fit(X_train[list(robust_features.keys())], y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [142]:
y_pred = model.predict(X_test[list(robust_features.keys())])

In [143]:
len(y_pred), len(y_test)

(1000, 1000)

In [144]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.61      0.44      0.51       378
        True       0.71      0.83      0.76       622

    accuracy                           0.68      1000
   macro avg       0.66      0.63      0.64      1000
weighted avg       0.67      0.68      0.67      1000



In [145]:
robust_features_names = [f for f in robust_features.keys()]

In [146]:
# make it easier to read the list of robust features (only show the ones with a very large coef -> likely to be important)
threshold = 1

pretty_robust_features = {k: v for k, v in robust_features.items() if abs(v['coefficient']) > threshold}
pretty_robust_features = {k: v['coefficient'] for k, v in sorted(pretty_robust_features.items(), key=lambda item: item[1]['coefficient'])}

pretty_robust_features_names = [f for f in pretty_robust_features.keys()]
pretty_robust_features

{'Notlw_Lasswell_nouns': -2.92866,
 'Know_GI': -2.52436,
 'Quan_GI_neg_3': -2.25218,
 'attention_neg_3': -1.58035,
 'Rsploss_Lasswell_adjectives': -1.52251,
 'Causal_GI_verbs': -1.3337,
 'If_Lasswell': -1.20076,
 'quality': -1.17408,
 'basic_nfunction_types': -1.16112,
 'Aquatic_GI_adjectives': -1.14377,
 'Passive_GI_verbs': -1.1242,
 'Emot_GI_verbs_neg_3': -1.12328,
 'hu_liu_prop_verbs': -1.1113,
 'Powoth_Lasswell': -1.08598,
 'Anxiety_GALC_adjectives': -1.03909,
 'Skloth_Lasswell_adjectives': -1.01587,
 'Submit_GI': -1.0014,
 'Say_GI_neg_3': 1.01424,
 'Ord_GI_adjectives_neg_3': 1.03378,
 'ADP': 1.0422,
 'Rcethic_Lasswell_verbs': 1.0821,
 'interactivity_2': 1.16675,
 'Tool_GI': 1.20594,
 'Persist_GI_nouns_neg_3': 1.24415,
 'NOUN': 1.25014,
 'Yes_GI': 1.28595,
 'argumentative': 1.38458,
 'Eval_GI_adverbs_neg_3': 1.41003,
 'VERB': 1.43709,
 'Powcoop_Lasswell_adverbs': 1.48853,
 'ttr': 1.53951,
 'Know_GI_verbs': 1.94893}

In [147]:
# save robust features dict (has features name, pvalue and vif for each)
with open(os.path.join(STATS_DIR, "robust_features_col-rfe.json"), 'w') as f:
    json.dump(robust_features, f, indent=4)

## RFE feature set

- more features (711)

1. Recursive Feature Elimination

In [148]:
data_df = load_x_y(
    features_filename=os.path.join(FEATURES_DIR, '12091031_all_features.csv.gz'),
    responses_filename=os.path.join(RESPONSES_DIR, '12091031_parsed_turbo_10000_eval.jsonl')
)
X_train, y_train, X_test, y_test = prepare_splits(data_df, test_size=0.2, random_state=1)

Instances: 10000
Features: 2371

Using existing splits
X_train shape: (8000, 2371)
y_train shape: (8000,)
X_test shape: (1000, 2371)
y_test shape: (1000,)


In [149]:
feature_set_source = os.path.join(STATS_DIR, "12091031_rfe_all_10000.json")
with open(feature_set_source) as f:
    stats = json.load(f)
    feature_set = [f for f in stats['coefficients'].keys()]

In [150]:
# Only use features in feature set
X_train = X_train[feature_set]
X_test = X_test[feature_set]

print('X_train shape:', X_train.shape, 'X_test shape:', X_test.shape)

X_train shape: (8000, 711) X_test shape: (1000, 711)


In [151]:
constant_features = X_train.columns[X_train.nunique() == 1] # constant features 

X_train = X_train.drop(columns=constant_features)
X_test = X_test.drop(columns=constant_features)

X_train.shape, X_test.shape

((8000, 709), (1000, 709))

In [152]:
# Check for multicollinearity - in RFE only setting, we have not alredy removed collinear features
corr_matrix = X_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

# keep copy of X_train and X_test with collinear features
X_train_with_corr = X_train.copy()
X_test_with_corr = X_test.copy()

# drop features in to_drop
X_train = X_train.drop(columns=to_drop)
X_test = X_test.drop(columns=to_drop)

len(to_drop)

177

In [153]:
coefficients = stats['coefficients']

threshold = 0.3
large_coefficients = {k: v for k, v in coefficients.items() if abs(v) > threshold}

In [154]:
len(coefficients), len(large_coefficients)

(711, 617)

In [155]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# calculate VIF and p-value

# a feature is robust if it has a low p-value and low VIF (and a large coefficient)
robust_features = {} # individual features and interactions

for feature, coef in large_coefficients.items():

    if feature in to_drop: # we've dropped these
        continue

    X = sm.add_constant(X_train[[feature]]) # stats models allows us to get the p-values
    X.reset_index(drop=True, inplace=True) # avoid index mismatch error
    y = y_train.reset_index(drop=True) # avoid index mismatch error
    
    sm_model = sm.Logit(y, X)
    result = sm_model.fit(method='lbfgs', maxiter=1000, disp=False)

    p_value = result.pvalues[1]
    
    vif = variance_inflation_factor(X.values, 1) # idx of col to check (0: constant, 1: feature since we add them one by one)

    if p_value < 0.05 and vif < 5: # robust and significant
        robust_features[feature] = {
            'coefficient': round(coef, 5), 
            'p_value': round(p_value, 5), 
            'vif': round(vif, 5)
        }

In [166]:
len(robust_features.keys())

176

In [157]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    penalty='l2', 
    C=1.0, 
    solver='lbfgs', 
    max_iter=1000, 
    random_state=1
)

model.fit(X_train[list(robust_features.keys())], y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [158]:
y_pred = model.predict(X_test[list(robust_features.keys())])

In [159]:
len(y_pred), len(y_test)

(1000, 1000)

In [160]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.63      0.47      0.53       378
        True       0.72      0.83      0.77       622

    accuracy                           0.69      1000
   macro avg       0.67      0.65      0.65      1000
weighted avg       0.69      0.69      0.68      1000



In [161]:
robust_features_names = [f for f in robust_features.keys()]

In [162]:
# make it easier to read the list of robust features (only show the ones with a very large coef -> likely to be important)
threshold = 0.8

pretty_robust_features = {k: v for k, v in robust_features.items() if abs(v['coefficient']) > threshold}
pretty_robust_features = {k: v['coefficient'] for k, v in sorted(pretty_robust_features.items(), key=lambda item: item[1]['coefficient'])}

pretty_robust_features_names = [f for f in pretty_robust_features.keys()]
pretty_robust_features

{'Negate_GI_nouns': -2.05529,
 'Notlw_Lasswell_nouns': -1.83287,
 'Arousal_nwords_adjectives': -1.27769,
 'Socrel_GI_verbs_neg_3': -1.2499,
 'Rel_GI_adjectives_neg_3': -1.22962,
 'Increas_GI_adjectives': -1.20249,
 'Role_GI_adjectives_neg_3': -1.19062,
 'Know_GI': -1.11482,
 'Econ_GI': -1.09186,
 'Quan_GI_adjectives': -1.05305,
 'Weak_GI_nouns_neg_3': -1.04468,
 'Ngtv_GI_adjectives': -1.03029,
 'Enltot_Lasswell_verbs': -1.02434,
 'attention': -1.01314,
 'Pleasur_GI_verbs': -1.01109,
 'Timespc_Lasswell_verbs': -1.00439,
 'hu_liu_neg_perc_nouns': -1.00075,
 'interactivity_0': -0.9578,
 'Skltot_Lasswell_adjectives_neg_3': -0.95241,
 'Ptlw_Lasswell_verbs': -0.94743,
 'No_GI_neg_3': -0.94025,
 'Endslw_Lasswell_adverbs_neg_3': -0.90483,
 'Powoth_Lasswell': -0.89708,
 'hu_liu_prop_verbs_neg_3': -0.89697,
 'Rsploss_Lasswell_adjectives': -0.89427,
 'If_Lasswell': -0.86375,
 'Surelw_Lasswell_verbs': -0.86285,
 'Com_GI_neg_3': -0.83167,
 'aptitude_nouns_neg_3': -0.82251,
 'Quan_GI': -0.80654,
 'A

#### Looking at features that are considered important across different sets

see `lr_all-features_results.ipynb`

- 'Notlw_Lasswell_nouns' appers in 5/5 feature sets and is robust

- 'Negate_GI_nouns' appears in 4/5 feature sets (all the ones without collinear features removal as the first step; here in RFE set) and is robust - has a moderate correlation with other features, but still acceptable

- 'Know_GI_neg_3' appears in 4/5 feature sets (all the ones without collinear features removal as the first step; here in RFE set) and is NOT robust. It is highly correlated with 'Know_GI'

see `features/all-features_high_correlations.json` for correlations across the full feature set (all extracted features)

In [163]:
coefficients['Know_GI_neg_3'], 'Know_GI_neg_3' in to_drop

# This feature is considered important by the model when used, but it is collinear with other features 
# (therefore not in col-rfe feature set)

(-1.6940323288559407, True)

In [164]:
corr_matrix = X_train_with_corr.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
upper['Know_GI_neg_3'][upper['Know_GI_neg_3'] > 0.70] # check correlation with other features

Know_GI    0.965055
Name: Know_GI_neg_3, dtype: float64

In [165]:
# save robust features dict (has features name, pvalue and vif for each)
with open(os.path.join(STATS_DIR, "robust_features_rfe.json"), 'w') as f:
    json.dump(robust_features, f, indent=4)